In [1]:
from ipywidgets import widgets, interact, fixed
from IPython.display import display
%matplotlib inline
import seaborn as sbn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp
from IPython.core.pylabtools import figsize
import scipy
import scipy.interpolate
from contextlib import redirect_stdout
from CoolProp import CoolProp
figsize(12, 10)
sbn.set_context("talk", font_scale=1)

In [2]:
pip install coolprop

Note: you may need to restart the kernel to use updated packages.


# Defining the function for the sensetivity analysis

In [45]:
@interact
def BV_Diff( L = 0.01, #...
                  tfinal = 21600, # ...
                  sat = 0.15,
                  i0 = 2.55,
                  T = 298., # ...
                  OV = -0.075, # ...
                 ):
    alpha = 2#...
    F = 96485#...
    R = 8.314#...
    current = np.array([0])
    O2_limit = 10**-20
#     OV = -0.075#
    CO20 = 0.26 #mol/m3
    COH0 = (10**-4)#mol/m3
    concO2 = np.array([0.26])
    concOH = np.array([10**-4])
    BC1 = CO20 #Boundary condition 1, in the bulk

    #   Diffusion
    N = 20  # number of points to discretize
    #L = 0.01
    X = np.linspace(0, L, N)  # position along the rod
    h = L / (N - 1)  # discretization spacing


    D = 2.9*10**-9  # Diffusivity

    #tfinal = 54000 #15 hours
    Ntsteps = tfinal
    dt = tfinal / (Ntsteps-1)
    t = np.linspace(0, tfinal, Ntsteps)
    alpha = D * dt / h**2


    # initial condition at t = 0
    C = np.zeros(X.shape)
    C[0] = BC1

    current = np.array([])

    for j in range(tfinal):
        N = np.zeros(C.shape)
        N[0] = BC1
        N[1:-1] = alpha * C[2:] + (1 - 2 * alpha) * C[1:-1] + alpha * C[0:-2]
        if N[-2] < sat: #to give it some time until the solution gets to a degree of saturation
            N[-1] = N[-2]  # derivative boundary condition flux = 0
            current = np.append(current, 0)
            concOH = np.append(concOH, concOH[0])
            concO2 = np.append(concO2, N[-1])

        else: #after the solution is saturated with O2
            #print('Saturated')
            iloc = i0*((concOH[j]*np.exp((alpha*F*OV)/(R*T))) - (N[-2]*np.exp((-(alpha)*F*OV)/(R*T)))) #Current density

            RO2 = -1*abs(iloc)/(F*4) #Rate of O2 consumption or flux at the surface 

            N[-1] = (RO2*h) + N[-2]  # derivative boundary condition flux = RO2, BC2
            new_concO2 = N[-1]
            concO2 = np.append(concO2, new_concO2)

            ROH = 4*abs(iloc)/(F*4) #Rate of OH production

            new_concOH = (ROH*h) + concOH[j]

            concOH = np.append(concOH, new_concOH)

            newCurrent = iloc*0.0104

            current = np.append(current, newCurrent)

        C[:] = N #Crucial for the loop
    
    return  concO2[-1]

interactive(children=(FloatSlider(value=0.01, description='L', max=0.03, min=-0.01), IntSlider(value=21600, de…

Let's see the interaction for pH change as well:

In [56]:
@interact
def BV_Diff( L = 0.01, #...
                  tfinal = 21600, # ...
                  sat = 0.15,
                  i0 = 2.55,
                  T = 298., # ...
                  OV = -0.075, # ...
                 ):
    alpha = 2#...
    F = 96485#...
    R = 8.314#...
    current = np.array([0])
    O2_limit = 10**-20
#     OV = -0.075#
    CO20 = 0.26 #mol/m3
    COH0 = (10**-4)#mol/m3
    concO2 = np.array([0.26])
    concOH = np.array([10**-4])
    BC1 = CO20 #Boundary condition 1, in the bulk

    #   Diffusion
    N = 20  # number of points to discretize
    #L = 0.01
    X = np.linspace(0, L, N)  # position along the rod
    h = L / (N - 1)  # discretization spacing


    D = 2.9*10**-9  # Diffusivity

    #tfinal = 54000 #15 hours
    Ntsteps = tfinal
    dt = tfinal / (Ntsteps-1)
    t = np.linspace(0, tfinal, Ntsteps)
    alpha = D * dt / h**2


    # initial condition at t = 0
    C = np.zeros(X.shape)
    C[0] = BC1

    current = np.array([])

    for j in range(tfinal):
        N = np.zeros(C.shape)
        N[0] = BC1
        N[1:-1] = alpha * C[2:] + (1 - 2 * alpha) * C[1:-1] + alpha * C[0:-2]
        if N[-2] < sat: #to give it some time until the solution gets to a degree of saturation
            N[-1] = N[-2]  # derivative boundary condition flux = 0
            current = np.append(current, 0)
            concOH = np.append(concOH, concOH[0])
            concO2 = np.append(concO2, N[-1])

        else: #after the solution is saturated with O2
            #print('Saturated')
            iloc = i0*((concOH[j]*np.exp((alpha*F*OV)/(R*T))) - (N[-2]*np.exp((-(alpha)*F*OV)/(R*T)))) #Current density

            RO2 = -1*abs(iloc)/(F*4) #Rate of O2 consumption or flux at the surface 

            N[-1] = (RO2*h) + N[-2]  # derivative boundary condition flux = RO2, BC2
            new_concO2 = N[-1]
            concO2 = np.append(concO2, new_concO2)

            ROH = 4*abs(iloc)/(F*4) #Rate of OH production

            new_concOH = (ROH*h) + concOH[j]

            concOH = np.append(concOH, new_concOH)

            newCurrent = iloc*0.0104

            current = np.append(current, newCurrent)

        C[:] = N #Crucial for the loop
     
    pH = 14 + np.log(concOH[tfinal] )
    pH0=14+np.log(COH0)
    pHChange=pH-pH0
    return  pHChange

interactive(children=(FloatSlider(value=0.01, description='L', max=0.03, min=-0.01), IntSlider(value=21600, de…

In [47]:
@interact
def BV_Diff( L = 0.01, #...
                  tfinal = 21600, # ...
                  sat = 0.15,
                  i0 = 2.55,
                  T = 298., # ...
                  OV = -0.075, # ...
                 ):
    alpha = 2#...
    F = 96485#...
    R = 8.314#...
    current = np.array([0])
    O2_limit = 10**-20
#     OV = -0.075#
    CO20 = 0.26 #mol/m3
    COH0 = (10**-4)#mol/m3
    concO2 = np.array([0.26])
    concOH = np.array([10**-4])
    BC1 = CO20 #Boundary condition 1, in the bulk

    #   Diffusion
    N = 20  # number of points to discretize
    #L = 0.01
    X = np.linspace(0, L, N)  # position along the rod
    h = L / (N - 1)  # discretization spacing


    D = 2.9*10**-9  # Diffusivity

    #tfinal = 54000 #15 hours
    Ntsteps = tfinal
    dt = tfinal / (Ntsteps-1)
    t = np.linspace(0, tfinal, Ntsteps)
    alpha = D * dt / h**2


    # initial condition at t = 0
    C = np.zeros(X.shape)
    C[0] = BC1

    current = np.array([])

    for j in range(tfinal):
        N = np.zeros(C.shape)
        N[0] = BC1
        N[1:-1] = alpha * C[2:] + (1 - 2 * alpha) * C[1:-1] + alpha * C[0:-2]
        if N[-2] < sat: #to give it some time until the solution gets to a degree of saturation
            N[-1] = N[-2]  # derivative boundary condition flux = 0
            current = np.append(current, 0)
            concOH = np.append(concOH, concOH[0])
            concO2 = np.append(concO2, N[-1])

        else: #after the solution is saturated with O2
            #print('Saturated')
            iloc = i0*((concOH[j]*np.exp((alpha*F*OV)/(R*T))) - (N[-2]*np.exp((-(alpha)*F*OV)/(R*T)))) #Current density

            RO2 = -1*abs(iloc)/(F*4) #Rate of O2 consumption or flux at the surface 

            N[-1] = (RO2*h) + N[-2]  # derivative boundary condition flux = RO2, BC2
            new_concO2 = N[-1]
            concO2 = np.append(concO2, new_concO2)

            ROH = 4*abs(iloc)/(F*4) #Rate of OH production

            new_concOH = (ROH*h) + concOH[j]

            concOH = np.append(concOH, new_concOH)

            newCurrent = iloc*0.0104

            current = np.append(current, newCurrent)

        C[:] = N #Crucial for the loop
     
    
    return  current[-1]

interactive(children=(FloatSlider(value=0.01, description='L', max=0.03, min=-0.01), IntSlider(value=21600, de…

# Sensitivity Analysis

In [59]:
from SALib.sample import morris as ms
from SALib.analyze import morris as ma
from SALib.plotting import morris as mp